# Imports

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # images
import numpy as np #numpy
import seaborn as sns #plotting
import tensorflow.compat.v2 as tf #use tensorflow v2 as a main 
import tensorflow.keras as keras # required for high level applications
from sklearn.model_selection import train_test_split # split for validation sets
from sklearn.metrics import accuracy_score, f1_score, classification_report # metrics
import scipy
import pandas as pd
import re

# Load data

In [ ]:
from sklearn.utils import shuffle

train_data = pd.read_csv('train_data_csfd.csv')
train_data = shuffle(train_data)
train_data.head()

,text,label
37105,"Viděl jsem jen první 2 minuty, když nohy němec...",0
37455,Tak tohle je dokonalost sama! Cameron je géniu...,1
12422,"Výbordelnej seriál, akorát mě udivuje to množs...",1
15031,Jedním slovem vyjímečné.,1
45823,"Tohle je možná vtip, ale ne moc vtipnej a doce...",0


In [ ]:
x_train = list(train_data['text'])
# print(x[:5])

y_train = list(train_data['label'])
# print(y[:5])

#x_train = x_train[:20000]
#y_train = y_train[:20000]

- uncomment to use other values

In [ ]:
test_data = pd.read_csv('test_data_csfd.csv')

x_test = list(test_data['text'])
y_test = list(test_data['label'])

# Train model and predict on test dataset

In [ ]:
from tensorflow import string as tf_string
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.compat.v1.keras.layers import CuDNNGRU, CuDNNLSTM
from tensorflow.keras.layers import LSTM, GRU, Bidirectional

# 1 - 5 Vectorizer parameters

In [ ]:
embedding_dim = 128 # Dimension of embedded representation
vocab_size = 30000 # Number of unique tokens in vocabulary
sequence_length = 50 # Output dimension after vectorizing 

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify=y_train)

## 1.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 50)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 50, 128)           3840000   
                                                                 
 lstm_2 (LSTM)               (None, 50, 64)            49408     
                                                                 
 gru_2 (GRU)                 (None, 50, 64)            24960     
                                                                 
 flatten_2 (Flatten)         (None, 3200)              0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 256
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
71/71 [==============================] - 37s 467ms/step - loss: 0.5092 - accuracy: 0.7295 - val_loss: 0.2958 - val_accuracy: 0.8740
Epoch 2/2
71/71 [==============================] - 28s 390ms/step - loss: 0.1867 - accuracy: 0.9277 - val_loss: 0.2626 - val_accuracy: 0.8920


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 7s 21ms/step - loss: 0.2813 - accuracy: 0.8785


[0.281250536441803, 0.8784999847412109]

In [ ]:
y_pred=model.predict(x_test)


accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 87.85% 
f1-score is 0.8777050830397584% 
              precision    recall  f1-score   support

           0       0.87      0.89      0.88      5000
           1       0.88      0.87      0.88      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



## 2.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 50)               0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 50, 128)           3840000   
                                                                 
 bidirectional (Bidirectiona  (None, 50, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 50, 128)          74496     
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 256
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
71/71 [==============================] - 63s 803ms/step - loss: 0.5167 - accuracy: 0.7282 - val_loss: 0.3100 - val_accuracy: 0.8670
Epoch 2/2
71/71 [==============================] - 71s 999ms/step - loss: 0.1846 - accuracy: 0.9285 - val_loss: 0.2817 - val_accuracy: 0.8845


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 10s 33ms/step - loss: 0.3076 - accuracy: 0.8706


[0.30757012963294983, 0.8705999851226807]

In [ ]:
y_pred=model.predict(x_test)


accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 87.06% 
f1-score is 0.8749033255993813% 
              precision    recall  f1-score   support

           0       0.90      0.84      0.87      5000
           1       0.85      0.91      0.87      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



## 3.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 50)               0         
 torization)                                                     
                                                                 
 embedding_3 (Embedding)     (None, 50, 128)           3840000   
                                                                 
 lstm_3 (LSTM)               (None, 50, 64)            49408     
                                                                 
 gru_3 (GRU)                 (None, 50, 64)            24960     
                                                                 
 flatten_3 (Flatten)         (None, 3200)              0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 256
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 6s 21ms/step - loss: 0.2422 - accuracy: 0.9000


[0.24216914176940918, 0.8999999761581421]

In [ ]:
y_pred=model.predict(x_test)


accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 90.0% 
f1-score is 0.8996789727126806% 
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      5000
           1       0.90      0.90      0.90      5000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



## 4.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 50)               0         
 ectorization)                                                   
                                                                 
 embedding_4 (Embedding)     (None, 50, 128)           3840000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 50, 128)          98816     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 50, 128)          74496     
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 256
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
178/178 [==============================] - 155s 833ms/step - loss: 0.3504 - accuracy: 0.8391 - val_loss: 0.2381 - val_accuracy: 0.9016
Epoch 2/2
178/178 [==============================] - 132s 741ms/step - loss: 0.1447 - accuracy: 0.9468 - val_loss: 0.2677 - val_accuracy: 0.8937


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 10s 33ms/step - loss: 0.2533 - accuracy: 0.8974


[0.2532559931278229, 0.8974000215530396]

In [ ]:
y_pred=model.predict(x_test)


accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 89.74% 
f1-score is 0.8963007883565798% 
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      5000
           1       0.91      0.89      0.90      5000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



## 5.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(emb)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Dropout(0.4)(x)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.4)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 50)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 50, 128)           3840000   
                                                                 
 bidirectional_4 (Bidirectio  (None, 50, 256)          263168    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 50, 128)          164352    
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 256
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
178/178 [==============================] - 281s 2s/step - loss: 0.3736 - accuracy: 0.8227 - val_loss: 0.2471 - val_accuracy: 0.8953
Epoch 2/2
178/178 [==============================] - 266s 1s/step - loss: 0.1580 - accuracy: 0.9429 - val_loss: 0.2554 - val_accuracy: 0.8943


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 21s 68ms/step - loss: 0.2503 - accuracy: 0.8976


[0.25027996301651, 0.897599995136261]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 89.75999999999999% 
f1-score is 0.8975180144115293% 
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      5000
           1       0.90      0.90      0.90      5000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



# 6 - 8 Vectorizer parameters

In [ ]:
embedding_dim = 128 # Dimension of embedded representation - this is already part of latent space, there is captured some dependecy among words, we are learning this vectors in ANN
vocab_size = 100000 # Number of unique tokens in vocabulary
sequence_length = 50 # Output dimension after vectorizing - words in vectorited representation are independent

vect_layer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_length)
vect_layer.adapt(x_train)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=69, stratify=y_train)

## 6.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = LSTM(64, activation='relu', return_sequences=True)(emb)
x = GRU(64, activation='relu', return_sequences=True)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 50)               0         
 ectorization)                                                   
                                                                 
 embedding_6 (Embedding)     (None, 50, 128)           12800000  
                                                                 
 lstm_7 (LSTM)               (None, 50, 64)            49408     
                                                                 
 gru_6 (GRU)                 (None, 50, 64)            24960     
                                                                 
 flatten_6 (Flatten)         (None, 3200)              0         
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 256
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
178/178 [==============================] - 86s 468ms/step - loss: 0.3849 - accuracy: 0.8191 - val_loss: 0.2422 - val_accuracy: 0.9036
Epoch 2/2
178/178 [==============================] - 82s 463ms/step - loss: 0.1293 - accuracy: 0.9530 - val_loss: 0.2422 - val_accuracy: 0.9073


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 6s 20ms/step - loss: 0.2491 - accuracy: 0.9008


[0.24905547499656677, 0.9007999897003174]

In [ ]:
y_pred=model.predict(x_test)


accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 90.08% 
f1-score is 0.9004815409309791% 
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      5000
           1       0.90      0.90      0.90      5000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



## 7.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(emb)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 50)               0         
 ectorization)                                                   
                                                                 
 embedding_7 (Embedding)     (None, 50, 128)           12800000  
                                                                 
 bidirectional_7 (Bidirectio  (None, 50, 128)          98816     
 nal)                                                            
                                                                 
 bidirectional_8 (Bidirectio  (None, 50, 128)          74496     
 nal)                                                            
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 256
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
178/178 [==============================] - 157s 840ms/step - loss: 0.3498 - accuracy: 0.8344 - val_loss: 0.2266 - val_accuracy: 0.9077
Epoch 2/2
178/178 [==============================] - 145s 813ms/step - loss: 0.1141 - accuracy: 0.9587 - val_loss: 0.2677 - val_accuracy: 0.9012


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 10s 32ms/step - loss: 0.2693 - accuracy: 0.8969


[0.26925718784332275, 0.8968999981880188]

In [ ]:
y_pred=model.predict(x_test)


accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 89.69% 
f1-score is 0.8952129281431039% 
              precision    recall  f1-score   support

           0       0.88      0.91      0.90      5000
           1       0.91      0.88      0.90      5000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



## 8.Experiment

In [ ]:
input_layer = keras.layers.Input(shape=(1,), dtype=tf_string)
x_v = vect_layer(input_layer)
emb = keras.layers.Embedding(vocab_size, embedding_dim)(x_v)
x = Bidirectional(LSTM(128, activation='relu', return_sequences=True))(emb)
x = Bidirectional(LSTM(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Dropout(0.4)(x)
x = Bidirectional(GRU(64, activation='relu', return_sequences=True))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, 'relu')(x)
x = keras.layers.Dense(32, 'relu')(x)
x = keras.layers.Dropout(0.4)(x)
output_layer = keras.layers.Dense(1, 'sigmoid')(x)

model = keras.Model(input_layer, output_layer)
model.summary()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (TextV  (None, 50)               0         
 ectorization)                                                   
                                                                 
 embedding_8 (Embedding)     (None, 50, 128)           12800000  
                                                                 
 bidirectional_9 (Bidirectio  (None, 50, 256)          263168    
 nal)                                                            
                                                                 
 bidirectional_10 (Bidirecti  (None, 50, 128)          164352    
 onal)                                                           
                                                           

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=70, restore_best_weights=True)

batch_size = 256
epochs = 2
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks=[es], epochs=epochs, batch_size=batch_size)

Epoch 1/2
178/178 [==============================] - 286s 2s/step - loss: 0.3762 - accuracy: 0.8228 - val_loss: 0.2199 - val_accuracy: 0.9119
Epoch 2/2
178/178 [==============================] - 275s 2s/step - loss: 0.1230 - accuracy: 0.9552 - val_loss: 0.2512 - val_accuracy: 0.9093


In [ ]:
model.evaluate(x_test,y_test)

313/313 [==============================] - 20s 65ms/step - loss: 0.2641 - accuracy: 0.9004


[0.2640919089317322, 0.9003999829292297]

In [ ]:
y_pred=model.predict(x_test)

accuracy_sc = accuracy_score(y_pred=y_pred.round(),y_true=y_test)*100
f1_sc = f1_score(y_pred=y_pred.round(),y_true=y_test)

print("Accuracy score is {}% ".format(accuracy_sc))
print("f1-score is {}% ".format(f1_sc))
print(classification_report(y_pred=y_pred.round(),y_true=y_test))

Accuracy score is 90.03999999999999% 
f1-score is 0.9020070838252656% 
              precision    recall  f1-score   support

           0       0.91      0.88      0.90      5000
           1       0.89      0.92      0.90      5000

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



# Save model

In [ ]:
model.save('CZ_FT_first')

INFO:tensorflow:Assets written to: FR_LSTM_sixth/assets


In [ ]:
!zip -r /content/CZFTfirst.zip /content/CZ_FT_first/

  adding: content/FR_LSTM_sixth/ (stored 0%)
  adding: content/FR_LSTM_sixth/variables/ (stored 0%)
  adding: content/FR_LSTM_sixth/variables/variables.index (deflated 67%)
  adding: content/FR_LSTM_sixth/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: content/FR_LSTM_sixth/assets/ (stored 0%)
  adding: content/FR_LSTM_sixth/keras_metadata.pb (deflated 90%)
  adding: content/FR_LSTM_sixth/saved_model.pb (deflated 78%)
